Recolectamos un par de funciones que ya hemos fabricados en laboratorios anteriores.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

class COOMat:
    nf = 0 # cantidad de filas
    nc = 0 # cantidad de columnas
    
    f = [] # indices de filas
    c = [] # indices de columnas
    e = [] # elementos
        
    def __init__(self,nf_,nc_,f_,c_,e_): # inicializar matriz de tamaño nf_ x nc_ con elementos
        self.nf=nf_
        self.nc=nc_
        self.f=f_
        self.c=c_
        self.e=e_
            
    def Ax(self,x):
        nnz = self.e.size
        y = np.zeros(self.nf)
        for j in range(nnz):
            y[self.f[j]] = y[self.f[j]] + self.e[j]*x[self.c[j]]
        
        return y
    
    def full(self):
        Afull = np.zeros((self.nf,self.nc))
        nnz = self.e.size
        for j in range(nnz):
            Afull[(self.f)[j],(self.c)[j]] = (self.e)[j]
        
        return Afull

Sea $\Omega=(0,1)^2$, $\mu:\Omega\rightarrow\mathbb{R}_{>0}$ una función positiva, y
$b:\Omega\rightarrow\mathbb{R}^2$ una función vectorial. El objetivo es aproximar la solución $u$ de la ecuación de Poisson de dos dimensiones:

\begin{align}
  \begin{split}
  -{\rm div}\;(\mu(x) \nabla u (x)) &= f(x) \text{ para todo } x\in\Omega,\\
  u(x) &= 0 \text{ para todo } x\in\partial\Omega
  \end{split}
\end{align}

con diferencias finitas. Para generar una malla, usamos los puntos $x_j = jh$, $j=0,\dots, n+1$, donde $n+1 = 1/h$, y $x_{ij} = (x_i,x_j)$. Definimos las mallas

\begin{align*}
  \Omega_h &:= \left\{ x_{ij} \mid 1 \leq i,j\leq n \right\},\\
  \overline\Omega_h &:= \left\{ x_{ij} \mid 0 \leq i,j\leq n+1 \right\},\\
  \partial\Omega_h &:=\overline\Omega_h \setminus\Omega_h.
\end{align*}

y hallamos $u_h:\overline\Omega_h\rightarrow\mathbb{R}$ dada por

\begin{align*}
  \begin{split}
  -{\rm div}_h\; (\mu(x_{ij}) \nabla_h u(x_{ij})) &= f(x_{ij}) \text{ para todo } x_{ij}\in\Omega_h,\\
  u_h(x_{ij}) &= 0 \text{ para todo } x_{ij}\in\partial\Omega_h,
  \end{split}
\end{align*}

donde

\begin{align*}
  \nabla_h u(x_{ij}) :=
  \begin{pmatrix}
    \frac{u(x_{ij})-u(x_{i-1,j})}{h}\\
    \frac{u(x_{ij})-u(x_{i,j-1})}{h}
  \end{pmatrix}
\end{align*}

y

\begin{align*}
  {\rm div}_h v(x_{ij}) := \frac{v_1(x_{i+1,j}) - v_1(x_{ij})}{h} + \frac{v_2(x_{i,j+1}) - v_2(x_{ij})}{h}.
\end{align*}

Si usamos la numeración *lexicográfica*

\begin{align}\label{mdf:2d:lex}
  \underline u_{h,(j-1) n+i}  = u_h(x_{ij}).
\end{align}

llegamos al sistema $A_h \underline u_h = \underline f_h$, donde $\underline f_{h,(j-1) n+i}  = f(x_{ij})$.

Explicite la discretización de

\begin{align*}
  \begin{split}
  -{\rm div}_h\; (\mu(x_{ij}) \nabla_h u(x_{ij})) 
  \end{split}
\end{align*}

y complete el código de abajo para que calcule la matriz $A_h$.

In [2]:
def stima2d(N,mu):
    nnz = N*(3*N-2) + 2*(N-1)*N
    f = np.zeros(nnz,dtype='int')
    c = np.zeros(nnz,dtype='int')
    e = np.zeros(nnz)
    
    # block diagonal
    for k in range(N):
        offset = k*(3*N-2) # number of elements already calculated
        
        # diagonal
        for j in range(N):
            f[offset + j] = k*N + j
            c[offset + j] = k*N + j
            e[offset + j] = ???  
        
        # superdiagonal
        for j in range(N-1):
            f[offset + N + j] = k*N + j
            c[offset + N + j] = k*N + j+1
            e[offset + N + j] = ???
        
        # subdiagonal
        for j in range(N-1):
            f[offset + 2*N-1 + j] = k*N + j+1
            c[offset + 2*N-1 + j] = k*N + j
            e[offset + 2*N-1 + j] = ???
            
    # block superdiagonal
    for k in range(N-1):
        offset = N*(3*N-2) + k*N # number of elements already calculated
        
        # diagonal
        for j in range(N):
            f[offset + j] = k*N + j
            c[offset + j] = k*N + j + N
            e[offset + j] = ???
            
    # block subdiagonal
    for k in range(N-1):
        offset = N*(3*N-2) + (N-1)*N + k*N  # number of elements already calculated
        
        # diagonal
        for j in range(N):
            f[offset + j] = k*N + j + N
            c[offset + j] = k*N + j
            e[offset + j] = ???
            
    e = (N+1)*(N+1)*e

    return COOMat(N*N,N*N,f,c,e)

SyntaxError: invalid syntax (2517439000.py, line 15)

La siguiente función calcula el vector $\underline f_h$.

In [3]:
def loadvec2d(N,ff):
    f = np.zeros(N*N)
    
    x = np.linspace(0,1,N+2)
    X,Y = np.meshgrid(x,x)
    
    for j in range(N):
        for k in range(N):
            f[j*N+k] = ff( X[j,k], Y[j,k] )
            
    return f

Verifique si su código calcula algo razonable:

In [ ]:
def f(x,y):
    return ???

def mu(x,y):
    return ???

N=100

b = loadvec2d(N,f)
A = stima2d(N,mu)
Afull = A.full()

u = np.linalg.solve(Afull,b)
uu = np.zeros((N+2,N+2))
uu[1:N+1,1:N+1] = u.reshape(N,N)
x = np.linspace(0,1,N+2)
X,Y = np.meshgrid(x,x)

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')

# Plot a 3D surface
ax.plot_surface(X, Y, uu,cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

ax.set_xlabel('X')
ax.set_xlim(0, 1)
ax.set_ylabel('Y')
ax.set_ylim(0, 1)
plt.show()